In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
sns.set_style('whitegrid')


# 01 - Importar DataFrames Originais

In [8]:
off = ".//Data//Small_Data//offline_sales.json"
on = './Data/Small_Data/online_orders.json'

In [9]:
## Carregar Arquivo (customer_id e visitor_id como string para evitar notação cientifica)
df_off = pd.read_json(off, lines=True, dtype={'customer_id': str})
df_on = pd.read_json(on, lines=True, dtype={'visitor_id':str,'customer_id': str})

In [10]:
## Substitui None por NaN
df_off.customer_id.replace('None', np.nan, inplace=True)
df_on.customer_id.replace('None', np.nan, inplace=True)
df_on.visitor_id.replace('None', np.nan, inplace=True)


## 1.1 - Comparação de Clientes que utilizam ambos serviços 
**Esses números descartam clientes com identificação faltante**

In [11]:
clients_off = df_off.dropna().groupby('customer_id').sum().reset_index().customer_id
clients_on = df_on.dropna().groupby('customer_id').sum().reset_index().customer_id

lista_off = list(clients_off)
lista_on = list(clients_on)

In [12]:
## Quantidade de clientes que compraram tanto online quanto offline
clients_off.isin(lista_on).sum()

457

In [13]:
#### Aprimorar essa função usando seleção por estado com maior faturamento
def get_state(name):
    return ''.join(list(df_off[df_off.customer_id==name].state.value_counts().index)[0])
    
    

In [14]:
## Cria lista de clients utilizaram ambos serviços
clients_both = []
estados = []
for name in lista_off:
    ## Se nome do cliente off existe no clients On apenda id, se nao, passsa
    if clients_on.isin([name]).sum()>=1:
        clients_both.append(name)
        estados.append(get_state(name))
    else:
        pass
    
    

In [15]:
dict_clients = dict(zip(clients_both, estados))

In [16]:
df_both = pd.DataFrame(list(zip(clients_both, estados)), columns=['customer_id','state'])

In [18]:
## Lista de Clientes Identificados que utilizaram ambos serviços
df_both.head()

,customer_id,state
0,30313235376530633063,RJ
1,30313239643866643564,RJ
2,30323636393364386334,RJ
3,30326138306138626565,RJ
4,30326337663562336538,RJ


In [19]:
#Numero de Clientes que utilizam ambos serviços
n_both = len(df_both)

# 02 - Importar Outputs Off e Online

In [20]:
out_off = './Outputs/Offline_Data/Off_Output_Horizontal.csv'
out_on = './Outputs/Online_Data/On_Output_Horizontal.csv'

In [21]:
o_off = pd.read_csv(out_off).drop('Unnamed: 0', axis=1)
o_on = pd.read_csv(out_on).drop('Unnamed: 0', axis=1)

## 2.1 - Gerar Dados de Saída Combinados

In [22]:
fat_off = int(o_off.faturamento_total.values)
fat_on = int(o_on.faturamento_total.values)
fat_total = fat_off+fat_on

In [23]:
print('Faturamento das Lojas Físicas:', fat_off)
print('Faturamento da Loja Virtual', fat_on)
print('Faturamento total:',fat_total)

Faturamento das Lojas Físicas: 13747940
Faturamento da Loja Virtual 5641837
Faturamento total: 19389777


# 03 - Importar Output PageView Visitors ID

In [24]:
f = './Outputs/Utils/visitors_grouped.csv'

In [27]:
visitors = pd.read_csv(f).drop('Unnamed: 0', axis=1)

## 3.1 - Manipulação 

In [62]:
## Gera uma lista contendo os ids dos visitantes que visualizaram paginas
lista_vis = list(visitors.visitor_id.unique())
len(lista_vis)

468581

In [68]:
## Verifica se todos dados de quem comprou está presente no DF visualização
df_on.visitor_id.isin(lista_vis).value_counts()

True     12169
False       68
Name: visitor_id, dtype: int64

**68 pessoas que compraram não estão na lista? Isso é possivel como?**  
  
**A principio esses 68 negativos existem devido à filtragem por tipo de página onde foi incluido apenas 'Product', adicionar pageType='Category' e verificar**

In [66]:
ids_bought = df_on[~df_on.visitor_id.isin(lista_vis)]

In [67]:
ids_bought

,date,visitor_id,deviceType,order_id,on_product_id,quantity,price,customer_id
28,2018-08-06,6533326563386363386365636,desktop,643663353463633,393437356562616,1,239,NaN
60,2018-08-09,6431343131393931326261383,desktop,613137393066636,393138353232346,1,279,38653236343631363863
128,2018-08-12,3235646631643530626338363,desktop,343966333265646,666465636562323,1,489,NaN
182,2018-08-15,6331613232303533653065643,desktop,633436633061346,633236343836623,1,559,37666665353532613732
330,2018-08-24,3531646138633635616361633,desktop,333633306462366,393637646437663,1,129,NaN
...,...,...,...,...,...,...,...,...
11290,2018-08-16,3166346666666461363836626,desktop,633734616433313,396335396565623,1,348,NaN
11345,2018-08-24,3861303637353463616530653,desktop,356665333564333,396464373131666,1,69,63623232386338653863
11351,2018-08-24,3531646138633635616361633,desktop,333633306462366,346465613433363,1,119,NaN
11992,2018-08-14,3236313533333530646462363,desktop,316638643962636,333437623263326,1,389,NaN
